# 

In [206]:
import selenium as sel
from selenium.webdriver.common.by import By
import pandas as pd
import time
import datetime
import base as bs
import ElTiempoFunciones as etf

import json
import re

import os as os
import csv

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [207]:
f = open('config.json')
empresas = json.load(f)

if not os.path.exists('../data/raw/eltiempo.csv'):
    os.makedirs('../data/raw/eltiempo.csv')
    
#for i in empresas['LaRepublica']:
    

In [208]:
#Empresa con la cual vamos a extraer los articulos
# TODO Tener una lista que itere por todos lo clientes
empresa = input("Digite la empresa a extraer: ")

In [209]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.eltiempo.com/buscar?q={empresa}')
# time.sleep(2)
driver.implicitly_wait(10) #Nueva metodología de wait

In [210]:
# El número total de ventanas que tiene la búsqueda

total_num_ventanas = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]/div[2]/div[2]/div/div[11]/div/ul/li[10]/a').get_attribute('href')
print(total_num_ventanas)

https://www.eltiempo.com/buscar/2768?q=ecopetrol


In [211]:
# # Recolecta datos hasta la ventana 20 del periódico El Tiempo
# total_num_ventanas = f'https://www.eltiempo.com/buscar/20?q={empresa}'

# Para las pruebas: Recolecta datos hasta la ventana 2 del periódico El Tiempo
total_num_ventanas = f'https://www.eltiempo.com/buscar/20?q={empresa}'

In [212]:
titulares=[]
get_url = driver.current_url
pag_actual = 1

while(get_url != total_num_ventanas):
        buscar = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]/div[2]/div[2]/div')
        articulos = buscar.find_elements(By.CLASS_NAME,"listing")

        for articulos in articulos:
                aux = articulos.find_element(By.XPATH, './/h3[contains(@class, "title-container")]')
                url = aux.find_element(By.XPATH,'.//a').get_attribute('href')
                # print(url)
                if not(bs.existedb(url, "eltiempo")):
                        titulo = articulos.find_element(By.CLASS_NAME,"title-container").text
                        # print(titulo)
                        resumen = articulos.find_element(By.CLASS_NAME,"epigraph-container").text
                        # print(resumen)
                        fechaPub = articulos.find_element(By.CLASS_NAME,"published-at").text
                        # print(fechaPub)
                        tema = articulos.find_element(By.CLASS_NAME,"category").text
                        # print(tema)
                        titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                        'Titulo':titulo,
                                        'Fecha Publicacion':fechaPub,
                                        'Tema':tema,
                                        'URL':url,
                                        'Resumen':resumen,
                                        'Empresa':empresa})

        pag_actual=pag_actual+1
        driver.get(f'https://www.eltiempo.com/buscar/{pag_actual}?q={empresa}')
        driver.implicitly_wait(10) # Wait mientras abre la ventana
        get_url = driver.current_url
# print(titulares)

In [213]:
buscar = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]/div[2]/div[2]/div')
articulos = buscar.find_elements(By.CLASS_NAME,"listing")

for articulos in articulos:
        if not(bs.existedb(url, "eltiempo")):
                titulo = articulos.find_element(By.CLASS_NAME,"title-container").text
                # print(titulo)
                resumen = articulos.find_element(By.CLASS_NAME,"epigraph-container").text
                # print(resumen)
                fechaPub = articulos.find_element(By.CLASS_NAME,"published-at").text
                # print(fechaPub)
                tema = articulos.find_element(By.CLASS_NAME,"category").text
                # print(tema)
                aux = articulos.find_element(By.XPATH, './/h3[contains(@class, "title-container")]')
                url = aux.find_element(By.XPATH,'.//a').get_attribute('href')
                # print(url)

                titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                'Titulo':titulo,
                                'Fecha Publicacion':fechaPub,
                                'Tema':tema,
                                'URL':url,
                                'Resumen':resumen,
                                'Empresa':empresa})

print(url)


https://www.eltiempo.com/economia/empresas/ecopetrol-se-seguiran-necesitando-las-regalias-e-impuestos-del-petroleo-735130


### Extrae info de cada URL

In [214]:
# Inicializa los cookies del navegador 
driver.delete_all_cookies()

# Busca cada articulo y las almacena en la lista de titulares
for tit in titulares:
    # clear all cookies in scope of session
    driver.get(tit['URL'])
    driver.implicitly_wait(10) #Nueva metodología de wait


    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException)
    contenido = ''
    try :
        html = WebDriverWait(driver,10,ignored_exceptions=ignored_exceptions).until(EC.presence_of_element_located((By.XPATH,'.//div[contains(@class,"modulos public-side")]')))
        parrafos = html.find_elements(By.XPATH,'.//p') 

    except:
        try:
            html = driver.find_element(By.XPATH,'.//div[contains(@class,"modulos public-side")]')
            parrafos = html.find_elements(By.XPATH,'.//p')
        except:
            contenido = 'SIN PARRAFOS'
        else:
            for i in parrafos:
                contenido += i.text

    else:
        try:
            for i in parrafos:
                contenido += i.text
        except:
            print('Error en el último for == el más dentro de "for tit in titulares:" externo')
    
    # agregar contenido al dict de titulares
    tit['Contenido'] = contenido
    driver.delete_all_cookies()
    
    #agregar lista de URLs de noticias relacionadas
    # tit['RelNewsUrls'] = bs.obtener_articulos_relacionados_eltiempo(driver)


error en el articulo con link a:  https://www.eltiempo.com/economia/sectores/petroleo-puede-el-turismo-reemplazarlo-en-colombia-en-el-corto-plazo-735424
error en el articulo con link a:  https://www.eltiempo.com/economia/sectores/reacciones-al-anuncio-de-ministra-de-minas-sobre-exploracion-petrolera-735468
error en el articulo con link a:  https://www.eltiempo.com/economia/sectores/colombia-exploracion-petrolera-efectos-economicos-de-frenar-contratos-735278
error en el articulo con link a:  https://www.eltiempo.com/podcast/el-primer-cafe/no-explorar-petroleo-gas-y-carbon-el-nuevo-reto-del-gobierno-nacional-735320


### Para agregar la info en un archivo CSV

In [215]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Resumen','URL','Empresa','Tema','Contenido']

# Para AGREGAR titulares a un archivo.
with open('../data/raw/eltiempo.csv', 'a', newline='', errors='ignore') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

In [216]:
# Imprimir los últimos datos de
titulares[-1]

# Cerrar navegador
driver.quit()